# Лабораторная работа 5. 
# Методы поиска подстроки в строке.


## Выполнил студент группы Самат уулу Максат БВТ2101
***

### Задание

1.  Реализовать методы поиска подстроки в строке.

2.  Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела.

3.  Реализовать возможность выбора опции чувствительности или нечувствительности к регистру.

4. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Задание №1:

Алгоритм Кнута-Морриса-Пратта | Упрощенный алгоритм Бойера-Мура

#### Задание №2:

Написать программу, определяющую, является ли данное расположение «решаемым», то есть можно ли из него за конечное число шагов перейти к правильному. Если это возможно, то необходимо найти хотя бы одно решение - последовательность движений, после которой числа будут расположены в правильном порядке.

### Выполнение:

In [4]:
import time
import random

import pandas as pd

#### Входные данные: 

In [18]:
target = "f"
text = "gucciflexflipflop"

In [13]:
text = input("Введите текст, в котором необходимо найти строку:")
target = input("Введите строку, которую необходимо найти в тексте:")

#### Задание №1:
***

##### Встроенный поиск Python

In [16]:
def python_search(target: str, text: str, reg=True):
    if not reg:
        text = text.lower()
        target = target.lower()
    
    return text.index(target)

In [19]:
print(python_search(target, text, reg=False))

5


##### Алгоритм Кнута-Морриса-Пратта

In [20]:
def kmp(target: str, text: str, reg=True):
    a = []

    if not reg:
        text = text.lower()
        target = target.lower()

    def prefix(s):
        p = [0] * len(s)
        for i in range(1, len(s)):
            k = p[i - 1]
            while k > 0 and s[k] != s[i]:
                k = p[k-1]
            if s[k] == s[i]:
                k += 1
            p[i] = k
        return p

    p = prefix(target + "#" + text)

    for i in range(len(text)):
        if p[len(target)+ i + 1] == len(target):
            a.append(i-len(target)+1)
            break

    return a

In [21]:
print(kmp(target, text))

[5]


##### Упрощенный алгоритм Бойера-Мура

In [11]:
def bmh(target: str, text: str, reg=True):
    a = []
    # Число символов в подстроке
    m = len(target) 
    # Число символов в строке
    n = len(text)

    if not reg:
        text = text.lower()
        target = target.lower()

    def shift_table(target: str, m: int):
        # Словарь смещений
        d = {}
        # Уникальные символы в подстроке
        unique_str = set()

        # Итерации с предпоследнего символа.
        for i in range(m-2, -1, -1):
            if target[i] not in unique_str:        
                d[target[i]] = m - i - 1
                unique_str.add(target[i])

        # Отдельно формируем последний символ.
        if target[m-1] not in unique_str:     
            d[target[m-1]] = m

        # Смещения для прочих символов
        d['*'] = m              

        return d
    
    d = shift_table(target, m)

    if n >= m:
        # Счетчик проверяемого символа в строке
        i = m - 1
        
        while i < n:
            k = 0
            j = 0
            fl_break = False
            for j in range(m-1, -1, -1):
                if text[i-k] != target[j]:
                    if j == m - 1:
                        # Смещение, если не равен последний символ подстроки.
                        off = d[text[i]] if d.get(text[i], False) else d['*'] 
                    else:
                        # Смещение, если не равен не последний символ подстроки.
                        off = d[target[j]]  

                    # Смещение счетчика строки
                    i += off    
                    # Если несовпадение символа, то flBreak = True.
                    fl_break = True  
                    break

                # Смещение для сравниваемого символа в строке
                k += 1

            # Если дошли до начала образа, значит, все его символы совпали.
            if not fl_break:
                a.append(i - k + 1)
                break

    return a

In [12]:
print(bmh(target, text))

[5]


#### Задание №2:
***

In [10]:
from heapq import heappop, heappush

def a_star(start_chain, goal_node):
    node_hash = {}
    chains_queue = []
    heappush(chains_queue, start_chain)

    while chains_queue:
        cur_chain = heappop(chains_queue)
        cur_node = cur_chain.last_node()

        if cur_node == goal_node:
            return cur_chain
        
        node_hash[cur_node] = cur_chain.g()

        for chain in cur_chain.get_neighbours():
            if chain.last_node() in node_hash:
                if chain.g() >= node_hash[chain.last_node()]:
                    continue

                node_hash[chain.last_node()] = chain.g()
                
            heappush(chains_queue, chain)

    return "No solution!"

In [9]:
def mhtn(a, b, n):
    return abs(a % n - b % n) + abs(a // n - b // n)

In [8]:
class chain15:
    def __str__(self):
        i = 0
        sstr = ""
        while i < self.size ** 2:
            sstr += str(self.board_state[i]) + " "
            if i % self.size == 3:
                sstr += "\n"
            i += 1
        return sstr

    def __init__(self, board_state, history=[]):
        self.board_state = list(board_state)
        self.size = int((len(board_state)**0.5))
        self.history = history
        self.quad_size = int(self.size * self.size)

    ## Эвристики

    # Сумма расстояний по строкам и столбцам от текущего 
    # расположения костяшки до ее правильной позиции.
    def manh_dst(self):
        dst = 0
        for i in range(0, self.quad_size):
            dst += mhtn((self.board_state[i] - 1) % self.quad_size, i, self.size)
        return int(dst)

    def last_node(self):
        return str(self.board_state)

    # Если текущая костяшка и костяшка рядом с ней находятся 
    # в местах друг друга, то мы должны подвинуть одну из 
    # костяшек в сторону, поэтому добавляем 2 к общей эвристике.
    def linear_conflict(self):
        conflict_count = 0
        return 2 * conflict_count

    def last_move(self):
        if self.board_state[-1] == self.quad_size - 1 or self.board_state[-1] == self.quad_size - self.size:
            return 0
        return 2
    
    # Если сосед текущей костяшки уже нходится на своём месте, 
    # то её придётся подвинуть, поэтому добавляем 2 к общей эвристике.
    def corner_tiles(self):
        conflict_count = 0
        # верхний левый угол
        if self.board_state[0] != 1:
            if self.board_state[1] == 2 or self.board_state[self.size] == self.size + 1:
                conflict_count += 1
        # верхний правый угол
        if self.board_state[self.size - 1] != self.size:
            if self.board_state[self.size - 2] == self.size - 1 or self.board_state[self.size * 2 - 1] == self.size * 2:
                conflict_count += 1
        # нижний левый угол
        if self.board_state[self.quad_size - self.size] != self.quad_size - self.size + 1:
            if self.board_state[self.quad_size - self.size * 2] == self.quad_size - self.size * 2 + 1 or self.board_state[self.quad_size - self.size + 1] == self.quad_size - self.size + 2:
                conflict_count += 1

        # правый нижний угол в эвристике не рассматривается, 
        # так как очень сложно скомбинировать этот случай с last_move.

        return 2 * conflict_count

    def simple_heur(self):
        dst = 0
        for i in range(0, int(self.quad_size)):
            if (self.board_state[i] - 1) != i:
                dst += 1
        return dst

    def h(self):
        return self.manh_dst() + self.last_move()

    def g(self):
        return len(self.history)

    def f(self):
        return self.h() + self.g()

    def __lt__(self, other):
        return self.f() < other.f()

    def get_neighbours(self):
        neighs = []
        zero_coord = self.board_state.index(0)

        # проверка соседей
        if zero_coord + 1 < self.size ** 2 and mhtn(zero_coord, zero_coord + 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + 1] = new_state[zero_coord + 1], new_state[zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord - 1 >= 0 and mhtn(zero_coord, zero_coord - 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - 1] = new_state[zero_coord - 1], new_state[zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord + self.size < self.size ** 2 and mhtn(zero_coord, zero_coord + self.size,
                                                                       self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + self.size] = new_state[zero_coord + self.size], new_state[
                zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord - self.size >= 0 and mhtn(zero_coord, zero_coord - self.size, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - self.size] = new_state[zero_coord - self.size], new_state[
                zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))


        return neighs

In [17]:
matrix = [5, 1, 9, 3, 
          11, 13, 6, 8, 
          14, 10, 4, 15,
          0, 12, 7, 2]

start = chain15(matrix)
end = chain15((1, 2, 3, 4, 
               5, 6, 7, 8, 
               9, 10, 11, 12, 
               13, 14, 15, 0))

result = a_star(start, end.last_node())
print(str(len(result.history)))
for node in result.history:
    print(str(node))
    print("-------------------------")
print(str(result))

41
5 1 9 3 
11 13 6 8 
14 10 4 15 
0 12 7 2 

-------------------------
5 1 9 3 
11 13 6 8 
0 10 4 15 
14 12 7 2 

-------------------------
5 1 9 3 
0 13 6 8 
11 10 4 15 
14 12 7 2 

-------------------------
5 1 9 3 
13 0 6 8 
11 10 4 15 
14 12 7 2 

-------------------------
5 1 9 3 
13 6 0 8 
11 10 4 15 
14 12 7 2 

-------------------------
5 1 0 3 
13 6 9 8 
11 10 4 15 
14 12 7 2 

-------------------------
5 0 1 3 
13 6 9 8 
11 10 4 15 
14 12 7 2 

-------------------------
5 6 1 3 
13 0 9 8 
11 10 4 15 
14 12 7 2 

-------------------------
5 6 1 3 
13 9 0 8 
11 10 4 15 
14 12 7 2 

-------------------------
5 6 1 3 
13 9 4 8 
11 10 0 15 
14 12 7 2 

-------------------------
5 6 1 3 
13 9 4 8 
11 10 7 15 
14 12 0 2 

-------------------------
5 6 1 3 
13 9 4 8 
11 10 7 15 
14 0 12 2 

-------------------------
5 6 1 3 
13 9 4 8 
11 0 7 15 
14 10 12 2 

-------------------------
5 6 1 3 
13 9 4 8 
11 7 0 15 
14 10 12 2 

-------------------------
5 6 1 3 
13 9 4 8 
11 7 15 0 
1

#### Тест скорости работы алгоритмов
***

In [7]:
def generate_text(n: int):
    string = [random.randint(32, 126) for _ in range(n)]
    result = ""
    for x in string:
        result += chr(x)
    
    return result

In [34]:
start, end, step = 50, 500, 50

results_kmp = []
results_bmh = []    
results_python_search = []

funlist = [kmp, bmh, python_search]
resultlist = [results_kmp, results_bmh, results_python_search]

for i in range(start, end, step):
    target_index_s = random.randint(0, i//2)
    target_index_e = random.randint(0, i//2)

    text = generate_text(i)
    target = text[target_index_s:target_index_s+target_index_e]

    for j in range(3):
        start_time = time.time()

        for _ in range(10000):
            funlist[j](target, text)
    
        end_time = time.time() - start_time
        resultlist[j].append(end_time)

data = { "Длина строки": [i for i in range(start, end, step)],
        "КМР": results_kmp,
        "Бойер-мур": results_bmh,
        "Python": results_python_search,
}

pd.set_option('display.width', 1000)
df = pd.DataFrame(data)
print(df)

   Длина строки       КМР  Бойер-мур    Python
0            50  1.871451   0.649689  0.029993
1           100  2.950595   1.200997  0.014000
2           150  3.838001   1.565000  0.028999
3           200  4.835000   1.783240  0.027004
4           250  5.685593   1.791001  0.015994
5           300  4.593001   1.562996  0.014002
6           350  3.650001   0.894000  0.016996
7           400  4.005021   0.888998  0.016002
8           450  6.111998   1.933028  0.016994


### Вывод